In [ ]:
import pandas as pd
from keplergl import KeplerGl
import json

# Load your latest merged data
df = pd.read_csv('merged_citibike_2022.csv')

/Users/sonaabraham/Documents/Career Foundry/Working - Demo/NYC-CitiBike-Analysis/citibike_env_new/lib/python3.11/site-packages/keplergl/keplergl.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string
/var/folders/gk/n7922c_x2k7f85761gxcw33w0000gn/T/ipykernel_1522/488735728.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_citibike_2022.csv')


Map Visualization Strategy

To visualize the flow of bike trips across New York City, I utilized Kepler.gl because of its ability to handle large-scale geospatial data efficiently.

Layer Choice (Arcs): I implemented an Arc Layer to connect starting and ending stations. Unlike flat lines, Arcs provide a 3D visualization of movement, which makes it easier to distinguish trip directionality and distance across the city's grid.

Color & Weight Configuration: The arcs are colored and weighted based on the trips variable. I chose a high-contrast gradient palette so that high-volume "power routes" stand out in bright colors, while infrequent trips remain subtle.

Why Kepler.gl?: While tools like Folium are good for static markers, Kepler allows for interactive filtering of millions of data points, which was essential for identifying the specific commuter corridors in this 2022 dataset.

In [ ]:
import pandas as pd
from keplergl import KeplerGl

# 1. Load your data (adjust filename if needed)
df = pd.read_csv('merged_citibike_2022.csv')

# 2. Create the 'value' column for counting
df['value'] = 1

# 3. Group by station and coordinates
# We keep lat/lng in the group so Kepler knows where to draw the arcs
df_group = df.groupby([
    'start_station_name', 'start_lat', 'start_lng', 
    'end_station_name', 'end_lat', 'end_lng'
])['value'].count().reset_index()

# 4. Rename for clarity
df_group.rename(columns={'value': 'trips'}, inplace=True)

print(f"Original rows: {len(df)}")
print(f"Aggregated trip paths: {len(df_group)}")

In [ ]:
# Create the map instance
m = KeplerGl(height=700, data={"NYC_Bike_Trips": df_group})

# Display the map
m

In [ ]:
# Filter for paths with more than 10 trips
df_filtered = df_group[df_group['trips'] >= 10]

# Now create the map with the smaller dataset
m = KeplerGl(height=700, data={"NYC_Bike_Trips": df_filtered})
m.save_to_html(file_name='NYC_CitiBike_Map_Optimized.html')

In [ ]:
# Define the default configuration for Arcs and 3D view
config = {
    'version': 'v1',
    'config': {
        'visState': {
            'layers': [{
                'type': 'arc',
                'config': {
                    'dataId': 'NYC_Bike_Trips',
                    'label': 'Trip Arcs',
                    'columns': {
                        'lat0': 'start_lat', 'lng0': 'start_lng', # Source
                        'lat1': 'end_lat', 'lng1': 'end_lng'      # Target
                    },
                    'visConfig': {
                        'opacity': 0.8,
                        'thickness': 2,
                        'colorRange': {
                            'name': 'Global Warming',
                            'type': 'sequential',
                            'colors': ['#5A1846', '#900C3F', '#C70039', '#FF5733', '#FFC300', '#F1C40F']
                        },
                        'sizeRange': [1, 10]
                    }
                }
            }],
            'filters': [{
                'dataId': ['NYC_Bike_Trips'],
                'id': 'trips_filter',
                'name': ['trips'],
                'type': 'range',
                'value': [10, 500] # Default filter for high-volume routes
            }]
        },
        'mapState': {
            'bearing': 0,
            'dragRotate': True,
            'pitch': 45, # Tilted view for 3D effect
            'zoom': 12,
            'latitude': 40.7128,
            'longitude': -74.0060
        }
    }
}

# Save the map with this config embedded
m.save_to_html(file_name='NYC_CitiBike_Final_Map.html', config=config)

Analysis of NYC Citi Bike Trip Patterns (2022)

After aggregating over 30 million rows of data and visualizing them with Kepler.gl, several clear geographical and behavioral patterns emerge:

Commuter "Last-Mile" Corridors: There is a significant density of trips originating and ending near major transit hubs such as Penn Station (8 Ave & W 31 St) and the Port Authority Bus Terminal (W 41 St & 8 Ave). This confirms that Citi Bikes serve as a vital link for commuters traveling from train stations to their final office destinations in Midtown.

High-Volume Recreational Hubs: Areas like Central Park West and the Hudson River Greenway show a high frequency of "loop" trips, where bikes are often returned to the same or nearby stations. This indicates heavy use by casual riders and tourists for leisure rather than point-to-point transit.

Inter-Borough Connectors: The map reveals "thick" arc clusters crossing the East River, specifically between North Brooklyn (stations like Broadway & Berry St) and the Lower East Side. These represent essential arteries for residents commuting between boroughs outside of the subway system.

Station Popularity: Using the filtering tool to isolate paths with over 100 trips highlights that the most popular individual routes are concentrated in high-density commercial zones, specifically along Broadway and 9th Avenue.

Geospatial Analysis and Findings

By applying a filter to the trips variable, I isolated the most common routes in the NYC Citi Bike network. My analysis reveals the following patterns:

The "Last Mile" Commute: Heavy trip density is concentrated around major transportation hubs like Penn Station and Grand Central Terminal. Research into NYC transit patterns confirms these hubs handle hundreds of thousands of commuters daily, many of whom use Citi Bike to reach their final office destinations in Midtown.

Recreational Arteries: There is a distinct, high-volume flow along the Hudson River Greenway and Central Park West. These are protected cycling paths, which additional research identifies as the most heavily utilized non-motorized routes in North America, catering to both tourists and local exercise enthusiasts.

Inter-Borough Connectors: Thick arc clusters are visible crossing the Williamsburg and Manhattan Bridges, representing a vital link for residents in Brooklyn who commute to the Lower East Side for work.

Least Busy Zones: Peripheral areas in upper Manhattan and the edges of the service area show significantly fewer arcs, suggesting these stations serve primarily as local neighborhood transit rather than major commuter arteries.

In [7]:
import pandas as pd
import numpy as np

# Load only what we NEED for the dashboard
input_csv = 'merged_citibike_2022.csv'
output_csv = 'reduced_data_to_plot_7.csv'
np.random.seed(32)

# Only keeping the essentials for the charts
cols_to_keep = ['start_station_name', 'date', 'avgTemp']

# Use a tiny 0.1% sample (0.001)
reader = pd.read_csv(input_csv, chunksize=500000, low_memory=False)
first_chunk = True

for chunk in reader:
    sample = chunk[np.random.rand(len(chunk)) <= 0.001]
    sample['value'] = 1
    # Check for lowercase consistency
    sample.columns = [c.lower() for c in sample.columns]
    
    # Filter to minimal columns
    sample_minimal = sample[['start_station_name', 'date', 'avgtemp', 'value']]
    
    # Write to file (Overwrite mode 'w' first to clear the old 111MB file)
    sample_minimal.to_csv(output_csv, mode='w' if first_chunk else 'a', index=False, header=first_chunk)
    first_chunk = False

print("File recreated. It should be very small now!")

/var/folders/gk/n7922c_x2k7f85761gxcw33w0000gn/T/ipykernel_1636/2262533979.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['value'] = 1
/var/folders/gk/n7922c_x2k7f85761gxcw33w0000gn/T/ipykernel_1636/2262533979.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['value'] = 1
/var/folders/gk/n7922c_x2k7f85761gxcw33w0000gn/T/ipykernel_1636/2262533979.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

File recreated. It should be very small now!


/var/folders/gk/n7922c_x2k7f85761gxcw33w0000gn/T/ipykernel_1636/2262533979.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['value'] = 1
